# Advanced Filtering Techniques for DAS Data

This notebook explores frequency-domain filtering methods for Distributed Acoustic Sensing (DAS) data processing.

## Overview

DAS data often contains various types of noise and interference that require sophisticated filtering approaches. This notebook covers:

### Topics Covered:
1. Spectral analysis and characterization
2. Advanced bandpass and notch filtering
3. Adaptive filtering techniques
4. Time-frequency analysis (STFT, Wavelet transforms)
5. Spectral whitening and balancing
6. Wiener filtering for noise reduction
7. Real-world scenarios and troubleshooting

### Learning Objectives:
- Understand frequency characteristics of DAS signals and noise
- Design and apply sophisticated filters
- Use time-frequency methods for non-stationary signals
- Optimize filter parameters for specific applications
- Handle challenging real-world noise scenarios

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, fft
from scipy.ndimage import median_filter
import warnings
warnings.filterwarnings('ignore')

# Set plotting parameters
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {signal.__version__ if hasattr(signal, '__version__') else 'N/A'}")

## 1. Synthetic DAS Data with Various Noise Types

We'll create realistic synthetic data containing:
- Signal of interest (seismic events)
- Ambient noise (broadband)
- Harmonic interference (power line noise)
- Coherent noise (traffic, machinery)
- Low-frequency drift

In [ ]:
def generate_das_data_with_noise(n_channels=200, n_samples=4000, fs=1000):
    """
    Generate synthetic DAS data with multiple noise types.
    
    Parameters:
    -----------
    n_channels : int
        Number of spatial channels
    n_samples : int
        Number of time samples
    fs : float
        Sampling frequency (Hz)
    
    Returns:
    --------
    data : ndarray
        Noisy DAS data
    clean_signal : ndarray
        Clean signal component
    time : ndarray
        Time axis
    """
    time = np.arange(n_samples) / fs
    channels = np.arange(n_channels)
    
    # Initialize arrays
    clean_signal = np.zeros((n_channels, n_samples))
    
    # 1. Signal of interest: Multiple seismic events
    event_times = [0.5, 1.5, 2.5]  # seconds
    event_freqs = [30, 45, 60]     # Hz
    
    for t0, f0 in zip(event_times, event_freqs):
        # Create Ricker wavelet
        t_wavelet = time - t0
        wavelet = (1 - 2*(np.pi*f0*t_wavelet)**2) * np.exp(-(np.pi*f0*t_wavelet)**2)
        
        # Add spatial coherence (plane wave)
        velocity = 2000  # m/s
        channel_spacing = 10  # m
        
        for i in range(n_channels):
            delay = i * channel_spacing / velocity
            delay_samples = int(delay * fs)
            
            if delay_samples < n_samples:
                shifted_wavelet = np.roll(wavelet, delay_samples)
                shifted_wavelet[:delay_samples] = 0
                clean_signal[i] += shifted_wavelet * np.exp(-i/100)  # Amplitude decay
    
    # 2. Ambient noise (broadband Gaussian)
    ambient_noise = np.random.randn(n_channels, n_samples) * 0.2
    
    # 3. Harmonic interference (power line at 50 Hz and harmonics)
    harmonic_noise = np.zeros((n_channels, n_samples))
    power_line_freqs = [50, 100, 150]  # Hz (fundamental + harmonics)
    power_line_amps = [0.3, 0.15, 0.08]
    
    for freq, amp in zip(power_line_freqs, power_line_amps):
        phase_variation = np.random.rand(n_channels) * 2 * np.pi
        for i in range(n_channels):
            harmonic_noise[i] += amp * np.sin(2*np.pi*freq*time + phase_variation[i])
    
    # 4. Coherent noise (traffic/machinery)
    coherent_noise = np.zeros((n_channels, n_samples))
    
    # Simulate passing vehicle
    vehicle_time = np.linspace(0.5, 2.0, 100)
    vehicle_freq = 20  # Hz (low frequency rumble)
    
    for i, t in enumerate(vehicle_time):
        t_idx = int(t * fs)
        if t_idx < n_samples:
            # Envelope
            envelope = np.exp(-((time - t)**2) / 0.02)
            
            # Add to nearby channels
            affected_channel = int(i * n_channels / len(vehicle_time))
            for ch in range(max(0, affected_channel-5), min(n_channels, affected_channel+5)):
                coherent_noise[ch] += 0.15 * envelope * np.sin(2*np.pi*vehicle_freq*time)
    
    # 5. Low-frequency drift
    drift = np.zeros((n_channels, n_samples))
    for i in range(n_channels):
        drift_freq = 0.5 + 0.2 * np.random.rand()  # 0.5-0.7 Hz
        drift[i] = 0.4 * np.sin(2*np.pi*drift_freq*time + np.random.rand()*2*np.pi)
    
    # Combine all components
    noisy_data = clean_signal + ambient_noise + harmonic_noise + coherent_noise + drift
    
    return noisy_data, clean_signal, time

# Generate data
noisy_data, clean_signal, time_axis = generate_das_data_with_noise(
    n_channels=200, n_samples=4000, fs=1000
)

print(f"Data shape: {noisy_data.shape}")
print(f"Time duration: {time_axis[-1]:.1f} s")
print(f"Signal-to-noise ratio: {10*np.log10(np.var(clean_signal)/np.var(noisy_data-clean_signal)):.1f} dB")

## 2. Spectral Analysis

Understanding the frequency content is crucial before designing filters.

In [ ]:
def analyze_spectrum(data, fs, channel_idx=None):
    """
    Compute and visualize power spectral density.
    
    Parameters:
    -----------
    data : ndarray
        Input data (n_channels x n_samples)
    fs : float
        Sampling frequency
    channel_idx : int or None
        Specific channel to analyze (None = average all)
    """
    if channel_idx is not None:
        trace = data[channel_idx]
    else:
        # Average spectrum across all channels
        trace = np.mean(data, axis=0)
    
    # Compute PSD using Welch's method
    freqs, psd = signal.welch(trace, fs=fs, nperseg=1024, 
                              noverlap=512, scaling='density')
    
    return freqs, psd

# Analyze spectra
freqs_noisy, psd_noisy = analyze_spectrum(noisy_data, fs=1000)
freqs_clean, psd_clean = analyze_spectrum(clean_signal, fs=1000)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Time domain - noisy
vmax = np.percentile(np.abs(noisy_data), 98)
im1 = axes[0, 0].imshow(noisy_data, aspect='auto', cmap='seismic',
                        extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                               noisy_data.shape[0], 0],
                        vmin=-vmax, vmax=vmax)
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Channel')
axes[0, 0].set_title('Noisy Data (Time Domain)', fontweight='bold')
plt.colorbar(im1, ax=axes[0, 0])

# Time domain - clean
vmax = np.percentile(np.abs(clean_signal), 98)
im2 = axes[0, 1].imshow(clean_signal, aspect='auto', cmap='seismic',
                        extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                               clean_signal.shape[0], 0],
                        vmin=-vmax, vmax=vmax)
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Channel')
axes[0, 1].set_title('Clean Signal (Time Domain)', fontweight='bold')
plt.colorbar(im2, ax=axes[0, 1])

# Frequency domain - linear scale
axes[1, 0].semilogy(freqs_noisy, psd_noisy, 'r-', linewidth=2, label='Noisy', alpha=0.7)
axes[1, 0].semilogy(freqs_clean, psd_clean, 'b-', linewidth=2, label='Clean', alpha=0.7)
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('Power Spectral Density')
axes[1, 0].set_title('Power Spectrum (Log Scale)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()
axes[1, 0].set_xlim([0, 200])

# Frequency domain - zoom on harmonics
axes[1, 1].plot(freqs_noisy, psd_noisy, 'r-', linewidth=2, label='Noisy')
axes[1, 1].plot(freqs_clean, psd_clean, 'b-', linewidth=2, label='Clean')
axes[1, 1].axvline(50, color='k', linestyle='--', alpha=0.5, label='50 Hz')
axes[1, 1].axvline(100, color='k', linestyle='--', alpha=0.5)
axes[1, 1].axvline(150, color='k', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('Power Spectral Density')
axes[1, 1].set_title('Harmonic Interference Detail', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()
axes[1, 1].set_xlim([0, 200])

plt.tight_layout()
plt.show()

print("\nSpectral characteristics identified:")
print("- Signal content: 20-80 Hz (seismic events)")
print("- Harmonic noise: 50, 100, 150 Hz (power line)")
print("- Low-frequency drift: < 2 Hz")
print("- Broadband ambient noise: all frequencies")

## 3. Advanced Bandpass Filtering

### Butterworth vs Chebyshev vs Elliptic Filters

Different filter designs offer trade-offs between:
- Passband flatness
- Stopband attenuation
- Transition width
- Phase linearity

In [ ]:
def design_filters(lowcut, highcut, fs, order=4):
    """
    Design different types of bandpass filters for comparison.
    
    Parameters:
    -----------
    lowcut, highcut : float
        Frequency bounds (Hz)
    fs : float
        Sampling frequency (Hz)
    order : int
        Filter order
    
    Returns:
    --------
    filters : dict
        Dictionary of filter coefficients
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    
    filters = {}
    
    # Butterworth (maximally flat passband)
    filters['butterworth'] = signal.butter(order, [low, high], btype='band')
    
    # Chebyshev Type I (sharper cutoff, passband ripple)
    filters['chebyshev1'] = signal.cheby1(order, 0.5, [low, high], btype='band')
    
    # Chebyshev Type II (sharper cutoff, stopband ripple)
    filters['chebyshev2'] = signal.cheby2(order, 40, [low, high], btype='band')
    
    # Elliptic (sharpest cutoff, both passband and stopband ripple)
    filters['elliptic'] = signal.ellip(order, 0.5, 40, [low, high], btype='band')
    
    return filters

def apply_filter_zerophase(data, filter_coefs):
    """
    Apply filter with zero phase distortion (forward-backward filtering).
    """
    b, a = filter_coefs
    filtered = np.zeros_like(data)
    
    for i in range(data.shape[0]):
        filtered[i] = signal.filtfilt(b, a, data[i])
    
    return filtered

# Design filters
filter_bank = design_filters(lowcut=15, highcut=80, fs=1000, order=4)

# Visualize filter responses
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

colors = ['blue', 'red', 'green', 'orange']
w = np.linspace(0, 500, 2000)  # Frequency points

for (name, (b, a)), color in zip(filter_bank.items(), colors):
    # Frequency response
    w_resp, h = signal.freqz(b, a, worN=w, fs=1000)
    
    # Magnitude
    ax1.plot(w_resp, 20*np.log10(np.abs(h)), color=color, 
             linewidth=2, label=name.capitalize())
    
    # Phase
    ax2.plot(w_resp, np.angle(h), color=color, linewidth=2, label=name.capitalize())

ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Magnitude (dB)')
ax1.set_title('Filter Magnitude Response', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xlim([0, 150])
ax1.axvline(15, color='k', linestyle='--', alpha=0.3)
ax1.axvline(80, color='k', linestyle='--', alpha=0.3)
ax1.axhline(-3, color='k', linestyle=':', alpha=0.3, label='-3 dB')

ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Phase (radians)')
ax2.set_title('Filter Phase Response', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_xlim([0, 150])

plt.tight_layout()
plt.show()

print("\nFilter characteristics:")
print("- Butterworth: Flattest passband, gentlest rolloff")
print("- Chebyshev I: Sharper rolloff, small passband ripple")
print("- Chebyshev II: Sharper rolloff, stopband ripple")
print("- Elliptic: Sharpest rolloff, ripple in both bands")
print("\nRecommendation: Butterworth for most seismic applications (linear phase)")

## 4. Notch Filtering for Harmonic Interference

Remove specific frequency components (e.g., power line noise) while preserving nearby frequencies.

In [ ]:
def design_notch_filter(f0, fs, Q=30):
    """
    Design a notch filter to remove a specific frequency.
    
    Parameters:
    -----------
    f0 : float
        Frequency to remove (Hz)
    fs : float
        Sampling frequency (Hz)
    Q : float
        Quality factor (higher = narrower notch)
        Typical values: 20-50
    
    Returns:
    --------
    b, a : ndarray
        Filter coefficients
    """
    b, a = signal.iirnotch(f0, Q, fs)
    return b, a

def apply_multi_notch(data, notch_freqs, fs, Q=30):
    """
    Apply multiple notch filters in cascade.
    """
    filtered = data.copy()
    
    for f0 in notch_freqs:
        b, a = design_notch_filter(f0, fs, Q)
        
        for i in range(filtered.shape[0]):
            filtered[i] = signal.filtfilt(b, a, filtered[i])
    
    return filtered

# Apply notch filters at 50, 100, 150 Hz
notch_freqs = [50, 100, 150]
notch_filtered = apply_multi_notch(noisy_data, notch_freqs, fs=1000, Q=30)

# Analyze result
freqs_notch, psd_notch = analyze_spectrum(notch_filtered, fs=1000)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Time domain before
vmax = np.percentile(np.abs(noisy_data), 98)
axes[0, 0].imshow(noisy_data, aspect='auto', cmap='seismic',
                  extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                         noisy_data.shape[0], 0],
                  vmin=-vmax, vmax=vmax)
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Channel')
axes[0, 0].set_title('Before Notch Filtering', fontweight='bold')

# Time domain after
axes[0, 1].imshow(notch_filtered, aspect='auto', cmap='seismic',
                  extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                         notch_filtered.shape[0], 0],
                  vmin=-vmax, vmax=vmax)
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Channel')
axes[0, 1].set_title('After Notch Filtering', fontweight='bold')

# Spectrum comparison
axes[1, 0].semilogy(freqs_noisy, psd_noisy, 'r-', linewidth=2, 
                    label='Before', alpha=0.7)
axes[1, 0].semilogy(freqs_notch, psd_notch, 'b-', linewidth=2, 
                    label='After', alpha=0.7)
for f in notch_freqs:
    axes[1, 0].axvline(f, color='k', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (log scale)')
axes[1, 0].set_title('Full Spectrum', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xlim([0, 200])

# Zoom on notches
axes[1, 1].plot(freqs_noisy, psd_noisy, 'r-', linewidth=2, label='Before', alpha=0.7)
axes[1, 1].plot(freqs_notch, psd_notch, 'b-', linewidth=2, label='After', alpha=0.7)
for f in notch_freqs:
    axes[1, 1].axvline(f, color='k', linestyle='--', alpha=0.5, linewidth=1.5)
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('PSD (linear scale)')
axes[1, 1].set_title('Notch Detail (40-160 Hz)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim([40, 160])

plt.tight_layout()
plt.show()

print(f"\n✓ Notch filtering applied at {notch_freqs} Hz")
print("\nPower line harmonics successfully attenuated!")

## 5. Time-Frequency Analysis

### Short-Time Fourier Transform (STFT)

For non-stationary signals, we need to understand how frequency content changes over time.

In [ ]:
def plot_spectrogram(data, fs, channel_idx=100, title="Spectrogram"):
    """
    Plot spectrogram using Short-Time Fourier Transform.
    
    Parameters:
    -----------
    data : ndarray
        Input data
    fs : float
        Sampling frequency
    channel_idx : int
        Channel to analyze
    """
    trace = data[channel_idx]
    
    # Compute STFT
    f, t, Sxx = signal.spectrogram(trace, fs=fs, nperseg=256, 
                                   noverlap=200, scaling='density')
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 6))
    
    im = ax.pcolormesh(t*1000, f, 10*np.log10(Sxx + 1e-10), 
                       shading='gouraud', cmap='viridis')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_xlabel('Time (ms)')
    ax.set_title(title, fontweight='bold')
    ax.set_ylim([0, 200])
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Power/Frequency (dB/Hz)')
    
    return fig, ax, f, t, Sxx

# Plot spectrograms
fig1, ax1, f1, t1, Sxx1 = plot_spectrogram(noisy_data, fs=1000, 
                                           title="Spectrogram: Noisy Data")
plt.show()

fig2, ax2, f2, t2, Sxx2 = plot_spectrogram(notch_filtered, fs=1000,
                                           title="Spectrogram: After Notch Filtering")
plt.show()

print("\nTime-frequency analysis shows:")
print("- Transient seismic events at ~0.5s, 1.5s, 2.5s")
print("- Persistent harmonic noise (horizontal lines at 50, 100, 150 Hz)")
print("- Coherent noise from passing vehicle (~0.5-2.0s)")
print("- Low-frequency drift (< 2 Hz)")

### Continuous Wavelet Transform (CWT)

Better time-frequency resolution for transient events.

In [ ]:
def analyze_with_cwt(data, fs, channel_idx=100):
    """
    Analyze signal using Continuous Wavelet Transform.
    """
    trace = data[channel_idx]
    
    # Define scales (inverse of frequencies)
    frequencies = np.logspace(0, 2.3, 100)  # 1 to ~200 Hz
    scales = fs / (2 * frequencies)
    
    # Compute CWT using Morlet wavelet
    coefficients = signal.cwt(trace, signal.morlet2, scales, w=6)
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 6))
    
    time = np.arange(len(trace)) / fs * 1000  # ms
    
    im = ax.pcolormesh(time, frequencies, np.abs(coefficients),
                       shading='gouraud', cmap='viridis')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_xlabel('Time (ms)')
    ax.set_title('Continuous Wavelet Transform', fontweight='bold')
    ax.set_yscale('log')
    ax.set_ylim([1, 200])
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Magnitude')
    
    return fig, ax

# Analyze clean signal with CWT
fig_cwt, ax_cwt = analyze_with_cwt(clean_signal, fs=1000)
plt.show()

print("\n✓ CWT provides excellent time-frequency localization")
print("- Clearly shows transient events at different frequencies")
print("- Superior to STFT for non-stationary signals")

## 6. Adaptive Filtering

### Time-Varying Bandpass Filter

Adapt filter parameters based on local signal characteristics.

In [ ]:
def adaptive_bandpass(data, fs, window_length=500, overlap=250):
    """
    Apply adaptive bandpass filtering based on local spectral content.
    
    Parameters:
    -----------
    data : ndarray
        Input data (n_channels x n_samples)
    fs : float
        Sampling frequency
    window_length : int
        Analysis window length (samples)
    overlap : int
        Overlap between windows (samples)
    
    Returns:
    --------
    filtered : ndarray
        Adaptively filtered data
    """
    n_channels, n_samples = data.shape
    filtered = np.zeros_like(data)
    
    step = window_length - overlap
    
    for ch in range(n_channels):
        trace = data[ch]
        filtered_trace = np.zeros(n_samples)
        weights = np.zeros(n_samples)
        
        # Process in windows
        for start in range(0, n_samples - window_length, step):
            end = start + window_length
            window = trace[start:end]
            
            # Compute local spectrum
            freqs, psd = signal.welch(window, fs=fs, nperseg=min(256, window_length))
            
            # Find dominant frequency range
            # Exclude very low and very high frequencies
            valid_range = (freqs > 10) & (freqs < 200)
            psd_valid = psd[valid_range]
            freqs_valid = freqs[valid_range]
            
            if len(psd_valid) > 0:
                # Find spectral centroid and spread
                centroid = np.sum(freqs_valid * psd_valid) / np.sum(psd_valid)
                spread = np.sqrt(np.sum(((freqs_valid - centroid)**2) * psd_valid) / 
                               np.sum(psd_valid))
                
                # Design adaptive filter
                lowcut = max(10, centroid - 1.5 * spread)
                highcut = min(200, centroid + 1.5 * spread)
                
                # Apply filter to window
                b, a = signal.butter(4, [lowcut/(fs/2), highcut/(fs/2)], btype='band')
                filtered_window = signal.filtfilt(b, a, window)
                
                # Apply taper for smooth transitions
                taper = signal.windows.tukey(window_length, alpha=0.2)
                
                # Accumulate
                filtered_trace[start:end] += filtered_window * taper
                weights[start:end] += taper
        
        # Normalize
        weights[weights == 0] = 1
        filtered[ch] = filtered_trace / weights
    
    return filtered

# Apply adaptive filtering
print("Applying adaptive bandpass filtering...")
adaptive_filtered = adaptive_bandpass(noisy_data, fs=1000, 
                                     window_length=500, overlap=250)

# Compare results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

titles = ['Original Noisy Data', 'Fixed Bandpass (15-80 Hz)', 'Adaptive Bandpass']

# Fixed bandpass for comparison
b, a = signal.butter(4, [15/500, 80/500], btype='band')
fixed_filtered = apply_filter_zerophase(noisy_data, (b, a))

datasets = [noisy_data, fixed_filtered, adaptive_filtered]

for ax, dataset, title in zip(axes, datasets, titles):
    vmax = np.percentile(np.abs(dataset), 98)
    ax.imshow(dataset, aspect='auto', cmap='seismic',
              extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                     dataset.shape[0], 0],
              vmin=-vmax, vmax=vmax)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Channel')
    ax.set_title(title, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Adaptive filtering complete")
print("\nAdvantages of adaptive approach:")
print("- Preserves varying frequency content")
print("- Better for non-stationary signals")
print("- Reduces over-filtering artifacts")

## 7. Spectral Whitening

Spectral whitening balances the frequency content, enhancing weak signals and suppressing dominant frequencies.

### Applications:
- Enhance weak reflections
- Normalize wavelet phase spectrum
- Improve cross-correlation (e.g., for ambient noise tomography)

In [ ]:
def spectral_whitening(data, fs, smooth_length=10):
    """
    Apply spectral whitening to normalize frequency content.
    
    Parameters:
    -----------
    data : ndarray
        Input data (n_channels x n_samples)
    fs : float
        Sampling frequency
    smooth_length : int
        Smoothing window for amplitude spectrum (prevents over-whitening)
    
    Returns:
    --------
    whitened : ndarray
        Spectrally whitened data
    """
    n_channels, n_samples = data.shape
    whitened = np.zeros_like(data)
    
    for i in range(n_channels):
        # FFT
        spectrum = fft.fft(data[i])
        amplitude = np.abs(spectrum)
        phase = np.angle(spectrum)
        
        # Smooth amplitude spectrum
        if smooth_length > 1:
            window = np.ones(smooth_length) / smooth_length
            amplitude_smooth = np.convolve(amplitude, window, mode='same')
        else:
            amplitude_smooth = amplitude
        
        # Prevent division by zero
        amplitude_smooth[amplitude_smooth < 1e-10] = 1e-10
        
        # Whiten: set all amplitudes to 1 (or normalized value)
        whitened_spectrum = (amplitude / amplitude_smooth) * np.exp(1j * phase)
        
        # Inverse FFT
        whitened[i] = np.real(fft.ifft(whitened_spectrum))
    
    return whitened

# Apply spectral whitening
whitened_data = spectral_whitening(clean_signal, fs=1000, smooth_length=10)

# Compare spectra
freqs_orig, psd_orig = analyze_spectrum(clean_signal, fs=1000)
freqs_white, psd_white = analyze_spectrum(whitened_data, fs=1000)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Time domain - original
vmax = np.percentile(np.abs(clean_signal), 98)
axes[0, 0].imshow(clean_signal, aspect='auto', cmap='seismic',
                  extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                         clean_signal.shape[0], 0],
                  vmin=-vmax, vmax=vmax)
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Channel')
axes[0, 0].set_title('Original Signal', fontweight='bold')

# Time domain - whitened
vmax = np.percentile(np.abs(whitened_data), 98)
axes[0, 1].imshow(whitened_data, aspect='auto', cmap='seismic',
                  extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                         whitened_data.shape[0], 0],
                  vmin=-vmax, vmax=vmax)
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Channel')
axes[0, 1].set_title('Spectrally Whitened', fontweight='bold')

# Frequency domain comparison
axes[1, 0].semilogy(freqs_orig, psd_orig, 'b-', linewidth=2, label='Original')
axes[1, 0].semilogy(freqs_white, psd_white, 'r-', linewidth=2, 
                    label='Whitened', alpha=0.7)
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (log scale)')
axes[1, 0].set_title('Power Spectral Density', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xlim([0, 200])

# Single trace comparison
ch_idx = 100
axes[1, 1].plot(time_axis*1000, clean_signal[ch_idx], 'b-', 
                linewidth=1.5, label='Original', alpha=0.7)
axes[1, 1].plot(time_axis*1000, whitened_data[ch_idx]*0.3, 'r-', 
                linewidth=1.5, label='Whitened (scaled)', alpha=0.7)
axes[1, 1].set_xlabel('Time (ms)')
axes[1, 1].set_ylabel('Amplitude')
axes[1, 1].set_title(f'Single Trace (Channel {ch_idx})', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Spectral whitening applied")
print("\nEffect: Flattened spectrum, enhanced high frequencies")
print("Use cases: Cross-correlation, weak signal enhancement")

## 8. Real-World Scenarios and Troubleshooting

### Scenario 1: Time-Varying Noise (Traffic Pattern)

**Problem**: Noise characteristics change over time (e.g., rush hour traffic)

**Solution**: Adaptive noise estimation and subtraction

In [ ]:
def adaptive_noise_suppression(data, fs, noise_estimate_window=0.5):
    """
    Estimate and subtract time-varying noise floor.
    
    Uses running minimum of spectral amplitude as noise estimate.
    """
    n_channels, n_samples = data.shape
    denoised = np.zeros_like(data)
    
    window_samples = int(noise_estimate_window * fs)
    
    for i in range(n_channels):
        # Compute STFT
        f, t, Zxx = signal.stft(data[i], fs=fs, nperseg=256, noverlap=200)
        
        # Estimate noise (running minimum in each frequency bin)
        magnitude = np.abs(Zxx)
        noise_estimate = np.zeros_like(magnitude)
        
        for freq_idx in range(len(f)):
            # Running minimum filter
            noise_estimate[freq_idx, :] = median_filter(
                magnitude[freq_idx, :], size=5, mode='reflect'
            ) * 0.8  # Slightly conservative
        
        # Spectral subtraction
        magnitude_clean = np.maximum(magnitude - noise_estimate, 0)
        
        # Reconstruct with original phase
        phase = np.angle(Zxx)
        Zxx_clean = magnitude_clean * np.exp(1j * phase)
        
        # Inverse STFT
        _, denoised[i] = signal.istft(Zxx_clean, fs=fs, nperseg=256, noverlap=200)
        
        # Ensure same length
        denoised[i] = denoised[i][:n_samples]
    
    return denoised

# Apply adaptive noise suppression
print("Applying adaptive noise suppression...")
denoised_data = adaptive_noise_suppression(noisy_data, fs=1000)

# Compare SNR
snr_original = 10*np.log10(np.var(clean_signal) / np.var(noisy_data - clean_signal))
snr_denoised = 10*np.log10(np.var(clean_signal) / np.var(denoised_data - clean_signal))

print(f"\nSNR improvement: {snr_denoised - snr_original:.1f} dB")
print(f"Original SNR: {snr_original:.1f} dB")
print(f"Denoised SNR: {snr_denoised:.1f} dB")

### Scenario 2: Fiber Coupling Issues

**Problem**: Spatially varying sensitivity due to coupling variations

**Solution**: Adaptive gain normalization (AGC)

In [ ]:
def automatic_gain_control(data, window_length=500, target_rms=1.0):
    """
    Apply automatic gain control to normalize amplitude variations.
    
    Parameters:
    -----------
    data : ndarray
        Input data
    window_length : int
        Window for RMS calculation (samples)
    target_rms : float
        Target RMS value
    """
    n_channels, n_samples = data.shape
    agc_data = np.zeros_like(data)
    
    for i in range(n_channels):
        trace = data[i]
        
        # Calculate running RMS
        rms = np.zeros(n_samples)
        for j in range(n_samples):
            start = max(0, j - window_length // 2)
            end = min(n_samples, j + window_length // 2)
            rms[j] = np.sqrt(np.mean(trace[start:end]**2))
        
        # Prevent division by zero
        rms[rms < 1e-10] = 1e-10
        
        # Apply gain
        gain = target_rms / rms
        
        # Limit maximum gain to prevent noise amplification
        gain = np.minimum(gain, 10.0)
        
        agc_data[i] = trace * gain
    
    return agc_data

# Simulate coupling variation
coupling_factor = np.ones(noisy_data.shape[0])
coupling_factor[50:100] = 0.3  # Poor coupling zone
coupling_factor[150:160] = 0.1  # Very poor coupling

data_with_coupling = noisy_data * coupling_factor[:, np.newaxis]

# Apply AGC
agc_corrected = automatic_gain_control(data_with_coupling, window_length=500)

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

titles = ['Original', 'With Coupling Variation', 'AGC Corrected']
datasets = [noisy_data, data_with_coupling, agc_corrected]

for ax, dataset, title in zip(axes, datasets, titles):
    vmax = np.percentile(np.abs(noisy_data), 98)
    ax.imshow(dataset, aspect='auto', cmap='seismic',
              extent=[time_axis[0]*1000, time_axis[-1]*1000, 
                     dataset.shape[0], 0],
              vmin=-vmax, vmax=vmax)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Channel')
    ax.set_title(title, fontweight='bold')
    
    if title == 'With Coupling Variation':
        ax.axhspan(50, 100, alpha=0.2, color='red', label='Poor coupling')
        ax.axhspan(150, 160, alpha=0.3, color='red')

plt.tight_layout()
plt.show()

print("\n✓ AGC successfully normalized amplitude variations")
print("\nWarning: Use AGC cautiously - can amplify noise in weak signal zones!")

## 9. Troubleshooting Guide

### Common Issues and Solutions

#### Issue 1: Over-filtering (Signal Distortion)

**Symptoms**:
- Signal appears smoothed or smeared
- Loss of high-frequency content
- Reduced temporal resolution

**Causes**:
- Filter cutoff too low
- Filter order too high
- Multiple cascaded filters

**Solutions**:
1. Use minimum necessary filter order (typically 2-4)
2. Verify filter response before applying
3. Compare filtered vs. unfiltered in frequency domain
4. Use zero-phase filtering (filtfilt) to preserve phase

#### Issue 2: Incomplete Noise Removal

**Symptoms**:
- Visible noise in filtered data
- Harmonic interference remains

**Causes**:
- Noise overlaps with signal band
- Insufficient notch Q-factor
- Time-varying noise characteristics

**Solutions**:
1. Increase notch Q-factor (30-50 typical)
2. Use adaptive methods for non-stationary noise
3. Consider F-K filtering for coherent noise
4. Apply spectral subtraction methods

#### Issue 3: Ringing Artifacts

**Symptoms**:
- Oscillations after sharp transitions
- Artificial events in filtered data

**Causes**:
- Sharp filter cutoffs (Chebyshev, Elliptic)
- High filter order
- Non-zero phase filtering

**Solutions**:
1. Use Butterworth filters (maximally flat)
2. Reduce filter order
3. Apply tapering to data edges
4. Use zero-phase filtering (filtfilt)

#### Issue 4: DAS-Specific: Gauge Length Effects

**Symptoms**:
- Spectral notches at specific frequencies
- Reduced amplitude at certain wavelengths

**Cause**: 
- DAS integrates strain over gauge length (typically 10m)
- Creates nulls at λ = gauge_length

**Solutions**:
1. Account for gauge length in frequency interpretation:
   - Notch frequency: f_notch = v / gauge_length
   - For v=3000 m/s, GL=10m: f_notch = 300 Hz
2. Cannot be corrected, but can be modeled
3. Choose appropriate gauge length during acquisition

#### Issue 5: Aliasing

**Symptoms**:
- High frequencies appear as low frequencies
- Unexplained low-frequency content

**Cause**:
- Sampling rate too low (fs < 2 * f_max)

**Solutions**:
1. Apply anti-aliasing filter before downsampling
2. Use lowpass with cutoff at 0.4 * fs (conservative)
3. Increase sampling rate if possible
4. Example:
```python
# Downsample from 1000 Hz to 500 Hz safely
b, a = signal.butter(8, 200/500, btype='low')  # 200 Hz < 0.5*500 Hz
filtered = signal.filtfilt(b, a, data)
downsampled = filtered[:, ::2]  # Decimate by 2
```

### Best Practices Summary

1. **Always visualize before and after filtering**
   - Time domain
   - Frequency domain
   - Time-frequency domain for non-stationary signals

2. **Start conservative**
   - Low filter orders (2-4)
   - Wide passbands
   - Gradually tighten if needed

3. **Use zero-phase filtering**
   - `filtfilt` instead of `lfilter`
   - Preserves phase relationships
   - Critical for multi-channel coherence

4. **Document all parameters**
   - Filter type and order
   - Cutoff frequencies
   - Processing sequence

5. **Quality control**
   - Check filter stability
   - Verify no artifacts introduced
   - Compare with independent data if available

6. **DAS-specific considerations**
   - Account for gauge length effects
   - Consider coupling variations (AGC)
   - Spatial filtering (F-K) for coherent noise

## Summary

This notebook covered advanced frequency-domain filtering techniques for DAS data:

### Key Concepts:

1. **Spectral Analysis**
   - Power spectral density (PSD)
   - Identifying signal vs. noise characteristics

2. **Filter Design**
   - Butterworth: Flat passband, smooth rolloff
   - Chebyshev: Sharp rolloff, with ripple
   - Elliptic: Sharpest rolloff, ripple in both bands
   - Notch: Remove specific frequencies

3. **Time-Frequency Methods**
   - STFT: Fixed time-frequency resolution
   - CWT: Adaptive resolution for transients

4. **Advanced Techniques**
   - Adaptive filtering: Time-varying parameters
   - Spectral whitening: Flatten spectrum
   - Adaptive noise suppression: STFT-based
   - AGC: Normalize spatial variations

### Practical Guidelines:

- **For seismic signals**: Butterworth bandpass (15-80 Hz typical)
- **For power line noise**: Cascade notch filters (50, 100, 150 Hz)
- **For non-stationary signals**: Adaptive or time-frequency methods
- **For coupling issues**: AGC with caution
- **For weak signals**: Spectral whitening

### Further Reading:

- Oppenheim & Schafer: "Discrete-Time Signal Processing" (classic text)
- Mallat: "A Wavelet Tour of Signal Processing" (wavelets)
- Yilmaz: "Seismic Data Analysis" (geophysical applications)

### Next Steps:

- Apply to real DAS field data
- Combine with spatial filtering (F-K, tau-p)
- Machine learning for adaptive noise classification
- Real-time filtering implementation